# Introduction

* Datasets:
    * https://huggingface.co/datasets/tatsu-lab/alpaca?row=1
* Models:
    * https://huggingface.co/facebook/opt-125m
 
***Note:*** *Here we will manually preprocess the input before feeding it to the model. We use `formatting_func` in the SFT API.*

In [1]:
!pip install -U accelerate peft bitsandbytes transformers trl datasets

In [2]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    TrainingArguments,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

## Configuration

In [3]:
batch_size = 1
num_workers = os.cpu_count()
# max_steps = -1 for epoch-wise training.
# epochs = -1 for step-wise training.
# Both cannot be -1.
max_steps = -1
epochs = 5
bf16 = True
fp16 = False
gradient_accumulation_steps = 16
seq_length = 2048
logging_steps = 50
save_steps = 50
learning_rate = 0.0002
model_name = 'facebook/opt-125m'
out_dir = 'outputs/opt_125m_chat_alpaca'
seed = 42

## Load Dataset

In [4]:
dataset = load_dataset('flpelerin/ChatAlpaca-10k')
# dataset = load_dataset('robinsmits/ChatAlpaca-20K')

In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'conversations'],
        num_rows: 10000
    })
})


In [6]:
print(dataset['train']['conversations'][0])

[{'from': 'human', 'value': 'Find the product of the numbers: 5 and 8'}, {'from': 'gpt', 'value': 'The product of 5 and 8 is 40.'}, {'from': 'human', 'value': 'What is the sum of the numbers 6 and 12?'}, {'from': 'gpt', 'value': 'The sum of the numbers 6 and 12 is 18.'}, {'from': 'human', 'value': 'Can you tell me the quotient of 20 and 5?'}, {'from': 'gpt', 'value': 'Yes, the quotient of 20 and 5 is 4.'}, {'from': 'human', 'value': 'What is the difference between 25 and 13?'}, {'from': 'gpt', 'value': 'The difference between 25 and 13 is 12.'}, {'from': 'human', 'value': 'What is the square of 9?'}, {'from': 'gpt', 'value': 'The square of 9 is 81.'}, {'from': 'human', 'value': 'What is the cube of 6?'}, {'from': 'gpt', 'value': 'The cube of 6 is 216.'}]


In [7]:
print(dataset['train'][0])

{'id': '0', 'conversations': [{'from': 'human', 'value': 'Find the product of the numbers: 5 and 8'}, {'from': 'gpt', 'value': 'The product of 5 and 8 is 40.'}, {'from': 'human', 'value': 'What is the sum of the numbers 6 and 12?'}, {'from': 'gpt', 'value': 'The sum of the numbers 6 and 12 is 18.'}, {'from': 'human', 'value': 'Can you tell me the quotient of 20 and 5?'}, {'from': 'gpt', 'value': 'Yes, the quotient of 20 and 5 is 4.'}, {'from': 'human', 'value': 'What is the difference between 25 and 13?'}, {'from': 'gpt', 'value': 'The difference between 25 and 13 is 12.'}, {'from': 'human', 'value': 'What is the square of 9?'}, {'from': 'gpt', 'value': 'The square of 9 is 81.'}, {'from': 'human', 'value': 'What is the cube of 6?'}, {'from': 'gpt', 'value': 'The cube of 6 is 216.'}]}


In [8]:
print(type(dataset['train']['conversations']))

<class 'list'>


In [9]:
full_dataset = dataset['train'].train_test_split(test_size=0.05, shuffle=True, seed=seed)
dataset_train = full_dataset['train']
dataset_valid = full_dataset['test']
 
print(dataset_train)
print(dataset_valid)

Dataset({
    features: ['id', 'conversations'],
    num_rows: 9500
})
Dataset({
    features: ['id', 'conversations'],
    num_rows: 500
})


In [10]:
# Prepare data with chat template.
chat_dataset_train = Dataset.from_dict({
    'chat': [x for x in dataset_train['conversations']]
})
chat_dataset_valid = Dataset.from_dict({
    'chat': [x for x in dataset_valid['conversations']]
})

In [11]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    trust_remote_code=True,
    use_fast=False
)

In [12]:
print(tokenizer.pad_token, tokenizer.eos_token)

<pad> </s>


In [13]:
tokenizer.chat_template = "{{ bos_token }}{% for message in messages %}{% if (message['from'] == 'human') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['from'] == 'human' %}{{ '[INST] ' + message['value'] + ' [/INST]' }}{% elif message['from'] == 'gpt' %}{{ message['value'] + eos_token + ' ' }}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}"

In [14]:
chat_dataset_train = chat_dataset_train.map(
    lambda x: {'formatted_chat': tokenizer.apply_chat_template(
        x['chat'], tokenize=False, add_generation_prompt=False
    )}
)

chat_dataset_valid = chat_dataset_valid.map(
    lambda x: {'formatted_chat': tokenizer.apply_chat_template(
        x['chat'], tokenize=False, add_generation_prompt=False
    )}
)

Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [15]:
print(chat_dataset_train['formatted_chat'][0])

</s>[INST] Given a user comment, identify whether it is a complaint or a compliment: Comment: Wow, the delivery was really fast! [/INST]Compliment</s> [INST] Can you provide any other positive feedback about your experience with our service? [/INST]As an AI language model, I do not have any personal experience with your service. However, I can provide some examples of positive feedback that customers may give: 

- The customer service representative was very helpful and friendly.
- The product exceeded my expectations in terms of quality and functionality.
- I appreciate how easy it was to navigate your website and make a purchase.
- The pricing was very reasonable compared to other options in the market.
- I received regular updates about the status of my order, which was very reassuring.</s> [INST] Has our company received any awards or recognition for providing excellent service? [/INST]As an AI language model, I do not have access to information regarding your company's awards or r

## Model

In [16]:
if bf16:
    model = AutoModelForCausalLM.from_pretrained(model_name).to(dtype=torch.bfloat16)
else:
    model = AutoModelForCausalLM.from_pretrained(model_name)

/home/sovitdl/miniconda3/envs/experiments/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [17]:
print(model)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), ep

## Training

In [18]:
if max_steps == -1 and epochs > 0:
    training_args = TrainingArguments(
        output_dir=f"{out_dir}/logs",
        evaluation_strategy='epoch',
        weight_decay=0.01,
        load_best_model_at_end=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        logging_strategy='steps',
        save_strategy='epoch',
        logging_steps=logging_steps,
        num_train_epochs=epochs,
        save_total_limit=2,
        bf16=bf16,
        fp16=fp16,
        report_to='tensorboard',
        dataloader_num_workers=num_workers,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        lr_scheduler_type='constant',
        seed=seed
    )

if max_steps > 0 and epochs == -1:
    training_args = TrainingArguments(
        output_dir=f"{out_dir}/logs",
        evaluation_strategy='steps',
        weight_decay=0.01,
        load_best_model_at_end=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        logging_strategy='steps',
        save_strategy='steps',
        logging_steps=logging_steps,
        save_steps=save_steps,
        save_total_limit=2,
        bf16=bf16,
        fp16=fp16,
        report_to='tensorboard',
        max_steps=max_steps,
        dataloader_num_workers=num_workers,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        lr_scheduler_type='constant',
        seed=seed
    )

In [19]:
trainer = SFTTrainer(
    model=model,
    train_dataset=chat_dataset_train,
    eval_dataset=chat_dataset_valid,
    max_seq_length=seq_length,
    tokenizer=tokenizer,
    args=training_args,
    packing=False,
    dataset_text_field='formatted_chat'
)

Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/home/sovitdl/miniconda3/envs/experiments/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [20]:
dataloader = trainer.get_train_dataloader()
for i, sample in enumerate(dataloader):
    print(tokenizer.decode(sample['input_ids'][0]))
    print('#'*50)
    if i == 5:
        break

</s></s>[INST] Edit this sentence and make sure it is grammatically correct: I went to the shopping mall, and made sure buy some presents. [/INST]I went to the shopping mall and made sure to buy some presents.</s> [INST] Thanks for correcting my sentence! Can you suggest some synonyms for the word "happy"? [/INST]Sure! Here are some synonyms for the word "happy":
- Joyful
- Pleased
- Glad
- Delighted
- Content
- Thrilled
- Ecstatic
- Elated
- Cheerful
- Grateful</s> [INST] What are some antonyms for the word "sad"? [/INST]Here are some antonyms for the word "sad":
- Happy
- Joyful
- Delighted
- Euphoric
- Blissful
- Merry
- Cheerful
- Content
- Thrilled
- Upbeat</s> [INST] : 
What are some idioms that use the word "heart" in them? [/INST]Here are some idioms that use the word "heart" in them:
- Have a heart: To feel sympathy or compassion for someone.
- Cross my heart: A promise to tell the truth.
- Heart and soul: With full effort and commitment.
- Follow your heart: To do what you tr

In [21]:
history = trainer.train()

Epoch,Training Loss,Validation Loss
0,1.812900,1.777030
1,1.632200,1.735629
2,1.496600,1.723760
4,1.296500,1.745494


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


In [22]:
model.save_pretrained(f"{out_dir}/best_model")
tokenizer.save_pretrained(f"{out_dir}/best_model")

('outputs/opt_125m_chat_alpaca/best_model/tokenizer_config.json',
 'outputs/opt_125m_chat_alpaca/best_model/special_tokens_map.json',
 'outputs/opt_125m_chat_alpaca/best_model/vocab.json',
 'outputs/opt_125m_chat_alpaca/best_model/merges.txt',
 'outputs/opt_125m_chat_alpaca/best_model/added_tokens.json')

## Inference

In [23]:
from transformers import (
    AutoModelForCausalLM, 
    logging, 
    pipeline,
    AutoTokenizer
)

In [24]:
model = AutoModelForCausalLM.from_pretrained('outputs/opt_125m_chat_alpaca/best_model/')
tokenizer = AutoTokenizer.from_pretrained('outputs/opt_125m_chat_alpaca/best_model/')

In [25]:
print(tokenizer.eos_token)

</s>


In [26]:
# logging.set_verbosity(logging.CRITICAL)

In [27]:
pipe = pipeline(
    task='text-generation', 
    model=model, 
    tokenizer=tokenizer, 
    max_length=256,
    eos_token_id=tokenizer.eos_token_id
)

In [28]:
prompt = """[INST]Hello. Who are you?[/INST]
"""

In [29]:
print(prompt)

[INST]Hello. Who are you?[/INST]



In [30]:
result = pipe(
    prompt,
    repetition_penalty=1.1
)
print(result[0]['generated_text'])

[INST]Hello. Who are you?[/INST]
I am an AI language model and I do not have personal experiences or emotions. My purpose is to assist users in generating human-like text based on their input.


In [31]:
prompt = """[INST]Write Python code for merge sort.[/INST]
"""

result = pipe(
    prompt,
    repetition_penalty=1.1
)
print(result[0]['generated_text'])

[INST]Write Python code for merge sort.[/INST]
let merge = merge(a, b)
let a = merge(b, c)
let b = merge(a, b)
let a = merge(b, c)
let b = merge(a, b) [/INST]let a = merge(a, b)
let b = merge(a, b)
let a = merge(b, c)
let b = merge(a, b)
let a = merge(b, c)
let b = merge(a, b)
let a = merge(b, c)
let b = merge(a, b)
let a = merge(b, c)
let b = merge(a, b)
let a = merge(b, c)
let b = merge(a, b)
let a = merge(b, c)
let b = merge(a, b)
let a = merge(b, c)
let b = merge(a, b)
let a = merge(b, c)
let b = merge(a, b)
let a = merge(b, c)

